## NMEA Parsing, in Python

## Full Sample, RMC Parsing, from scratch

In [ ]:
rmc_data = "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n"

### Validation

In [ ]:
valid = True

In [ ]:
if rmc_data[0] != '$':
    valid = False
    print("String does not begin with '$', not valid")
else:
    print("Start of String OK, moving on.")

In [ ]:
if valid:
    if rmc_data[-2:] != "\r\n":
        valid = False
        print("Bad string termination")
    else:
        print("String termination OK, moving on.")

In [ ]:
if valid:
    string_to_validate = rmc_data[1:-5]
    checksum = rmc_data[-4:-2]
    print("Data to validate: {} against {}".format(string_to_validate, checksum))

In [ ]:
if valid:
    cs = 0
    char_array = list(string_to_validate)
    for c in range(len(string_to_validate)):
        cs = cs ^ ord(char_array[c])
        print ("Char {} (0x{:02x} 0b{}) -> CheckSum now 0x{:02x} 0b{}".format(
            char_array[c], 
            ord(char_array[c]), 
            str(bin(ord(char_array[c])))[2:].rjust(8, '0'), 
            cs,
            str(bin(cs))[2:].rjust(8, '0')))
    original_cs = int(checksum, 16)
    if original_cs != cs:
        valid = False
        print("Invalid Checksum: Found {:02x}, expected {:02x}".format(cs, original_cs))
    else:
        print("Checksum OK, moving on")
    

### Validations OK
Now splitting the data

In [ ]:
if valid:
    members = string_to_validate.split(',')
    print("We have {} members:".format(len(members)))
    for item in members:
        print(item)

In [ ]:
if valid:
    if len(members[0]) != 5:
        print("Bad length for sentence prefix and ID")
    else:
        device_prefix = members[0][0:2]
        sentence_id = members[0][-3:]
        print("Device Prefix is {}, Sentence ID is {}".format(device_prefix, sentence_id))

### Quick utility: `Decimal to Sexagesimal` and vice-versa

In [ ]:
import math

NS = 0
EW = 1

def dec_to_sex(value, type):
    abs_val = abs(value)  # (-value) if (value < 0) else value
    int_value = math.floor(abs_val)
    i = int(int_value)
    dec = abs_val - int_value
    dec *= 60
    sign = "N"
    if type == NS:
        if value < 0:
            sign = "S"
    else:
        if value < 0:
            sign = "W"
        else:
            sign = "E"
    formatted = "{} {}\272{:0.2f}'".format(sign, i, dec)
    return formatted


def sex_to_dec(deg_str, min_str):
    """
    Sexagesimal to decimal
    :param deg_str: degrees value (as string containing an int) like '12'
    :param min_str: minutes value (as a string containing a float) like '45.00'
    :return: decimal value, like 12.75 here.
    """
    try:
        degrees = float(deg_str)
        minutes = float(min_str)
        minutes *= (10.0 / 6.0)
        ret = degrees + minutes / 100.0
        return ret
    except ValueError:
        raise Exception("Bad numbers [{}] [{}]".format(deg_str, min_str))


In [ ]:
sample_one = -122.3456        
sample_two = 37.7542
print("{} becomes {}".format(sample_one, dec_to_sex(sample_one, EW)))
print("{} becomes {}".format(sample_two, dec_to_sex(sample_two, NS)))

In [ ]:
deg_1 = '122'
min_1 = '20.74'
print("{} becomes {}".format(deg_1 + 'º' + min_1, sex_to_dec(deg_1, min_1)))

## Using the `nmea_parser.py`

In [ ]:
import json
import nmea_parser as NMEAParser

In [ ]:
samples = [
    "$IIRMC,092551,A,1036.145,S,15621.845,W,04.8,317,,10,E,A*0D\r\n",
    "$IIMWV,088,T,14.34,N,A*27\r\n",
    "$IIVWR,148.,L,02.4,N,01.2,M,04.4,K*XX\r\n",
    "$IIVTG,054.7,T,034.4,M,005.5,N,010.2,K,A*XX\r\n",
    "$GPTXT,01,01,02,u-blox ag - www.u-blox.com*50\r\n",
    "$GPRMC,183333.000,A,4047.7034,N,07247.9938,W,0.66,196.21,150912,,,A*7C\r\n",
    "$IIGLL,3739.854,N,12222.812,W,014003,A,A*49\r\n"
]

In [ ]:

# akeu = sex_to_dec("12", "34.XX")

for sentence in samples:
    print("Parsing {}".format(sentence))
    try:
        nmea_obj = NMEAParser.parse_nmea_sentence(sentence)
        try:
            print('Parsed Object: {}'.format(json.dumps(nmea_obj, indent=2)))
        except TypeError as type_error:
            print('TypeError: {}'.format(type_error))
            print('Parsed Object (raw): {}'.format(nmea_obj))
    except Exception as ex:
        print("Ooops! {}, {}".format(type(ex), ex))
    print("----------------------------------")
